In [10]:

import stackstac
import xarray as xr
import numpy as np
import geopandas as gpd
import pystac_client
import planetary_computer

In [ ]:
# connect to planetary computer stac catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [3]:
# list all collections within the catalog
collections = catalog.get_all_collections()
for collection in collections:
    print(f"{collection.id}: {collection.title}")

daymet-annual-pr: Daymet Annual Puerto Rico
daymet-daily-hi: Daymet Daily Hawaii
3dep-seamless: USGS 3DEP Seamless DEMs
3dep-lidar-dsm: USGS 3DEP Lidar Digital Surface Model
fia: Forest Inventory and Analysis
sentinel-1-rtc: Sentinel 1 Radiometrically Terrain Corrected (RTC)
gridmet: gridMET
daymet-annual-na: Daymet Annual North America
daymet-monthly-na: Daymet Monthly North America
daymet-annual-hi: Daymet Annual Hawaii
daymet-monthly-hi: Daymet Monthly Hawaii
daymet-monthly-pr: Daymet Monthly Puerto Rico
gnatsgo-tables: gNATSGO Soil Database - Tables
hgb: HGB: Harmonized Global Biomass for 2010
cop-dem-glo-30: Copernicus DEM GLO-30
cop-dem-glo-90: Copernicus DEM GLO-90
terraclimate: TerraClimate
nasa-nex-gddp-cmip6: Earth Exchange Global Daily Downscaled Projections (NEX-GDDP-CMIP6)
gpm-imerg-hhr: GPM IMERG
gnatsgo-rasters: gNATSGO Soil Database - Rasters
3dep-lidar-hag: USGS 3DEP Lidar Height above Ground
io-lulc-annual-v02: 10m Annual Land Use Land Cover (9-class) V2
goes-cmi: GOE

In [19]:
# search the catalog for S2 imagery
gdf = gpd.read_file("aoi.geojson") #aoi
dates = "2024-07-01/2024-08-31" #start/end dates


search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=gdf.total_bounds,
    datetime=dates,
    query={"eo:cloud_cover": {"lt": 1}},
)

items = search.item_collection()
print(f"Returned {len(items)} Items")

Returned 60 Items


In [ ]:
# create a stack of retrieved items
ds = stackstac.stack(
    items,
    assets=["B02", "B03", "B04", "B8A"],
    resolution=10,
    epsg=32609,
    bounds_latlon=gdf.total_bounds,
    chunksize=2048,
)
ds

<xarray.DataArray 'stackstac-3b89caefe9756712300a6078350d3d49' (time: 60,
                                                                band: 4,
                                                                y: 21737,
                                                                x: 22001)>
dask.array<fetch_raster_window, shape=(60, 4, 21737, 22001), dtype=float64, chunksize=(1, 1, 2048, 2048), chunktype=numpy.ndarray>
Coordinates: (12/43)
  * time                                     (time) datetime64[ns] 2024-07-05...
    id                                       (time) <U54 'S2A_MSIL2A_20240705...
  * band                                     (band) <U3 'B02' 'B03' 'B04' 'B8A'
  * x                                        (x) float64 7.859e+05 ... 1.006e+06
  * y                                        (y) float64 5.566e+06 ... 5.349e+06
    sat:relative_orbit                       (time) int32 13 13 99 ... 99 99 99
    ...                                       ...
    gsd                                      (band) float64 10.0 10.0 10.0 20.0
    title                                    (band) <U37 'Band 2 - Blue - 10m...
    common_name                              (band) <U7 'blue' ... 'rededge'
    center_wavelength                        (band) float64 0.49 0.56 ... 0.865
    full_width_half_max                      (band) float64 0.098 ... 0.033
    epsg                                     int32 32609
Attributes:
    spec:        RasterSpec(epsg=32609, bounds=(785920, 5349000, 1005930, 556...
    crs:         epsg:32609
    transform:   | 10.00, 0.00, 785920.00|\n| 0.00,-10.00, 5566370.00|\n| 0.0...
    resolution:  10